# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install gradio==6.1.0 openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. LangChain 里的记忆机制（Memory）

在对话场景中，记忆（Memory） 是一个非常关键的能力。它让大模型能够“记住”之前的对话内容，从而在后续回答中保持上下文连贯。

## 2.1 为什么需要记忆？

大模型本质上是“无状态”的，每次调用时，它只根据输入上下文生成输出。为了让对话看起来“连续”，我们需要额外的机制来保存历史记录并在后续调用中传递：

* **上下文连贯**：用户前面提到的人物、地点、问题，在后续对话里依然能被理解。
* **个性化记忆**：保存用户的偏好或背景信息，以便做出个性化回答。
* **任务跟踪**：在多轮对话中持续推进任务，不需要用户重复输入。


比如下面的例子，我们与模型进行多轮的对话后，由于我们没有配置和记忆相关的内容，因此其还是记不住我们第一次说过的名称信息。所以如果没有 Memory，大模型只能看到最后一句，无法理解上下文。而如果用了 Memory，大模型可以看到前面所有内容，就可以做出更有逻辑的回答。

In [3]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
  model="ernie-3.5-8k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3" 
)

response1 = llm.invoke("我叫李，你是谁？")
response2 = llm.invoke("你住在哪？")
response3 = llm.invoke("还记得我叫啥不")

print(response3.content)

我们才刚开始对话，我还不清楚您的名字呢。不过，如果您愿意告诉我，我会很乐意记住并称呼您的！您希望我怎么称呼您呢？


那下面我们就来介绍一下 LangChain 里常常能够用到的记忆类型吧！

## 2.2 全量记忆

### 2.2.1 ConversationBufferMemory（旧版全量记忆）

#### 概念

早期的 LangChain 使用 **ConversationBufferMemory** 来保存对话历史。它会完整保存用户与模型之间的所有对话历史，包括每一次提问（user message）和每一次回答（AI response），并在每次调用语言模型时将这些内容作为上下文重新发送，确保模型能“记住”之前发生过的所有交流。


它的原理很简单：

* 每次对话会追加进一个“对话缓存”列表。
* 调用模型时，LangChain 会把缓存中的所有内容拼接起来放入 prompt。
* 所以模型每次响应时，都会拥有“完整上下文”。

#### 存储格式

一般而言， **ConversationBufferMemory** 会将所有的对话信息保存在 `memory.chat_memory.messages` ，这样一个列表里面。里面包含了所有对话的信息，包括 `HumanMessage` 、 `AIMessage` 以及 `SystemMessage` 等等。比如像下面这样的例子：

```python 
memory.chat_memory.messages = [
  HumanMessage(content="你好"),
  AIMessage(content="你好！我可以帮你什么？"),
  HumanMessage(content="1+1等于几"),
  AIMessage(content="等于2"),
  ...
]
```

#### 代码示例


In [4]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory
import os

llm = ChatOpenAI(
model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, 
  memory=memory,
  verbose=True)
  
response_1 = conversation.predict(input="你好，我叫李剑锋")
print("对话 1:", response_1) 
response_2 = conversation.predict(input="请问1+1等于多少?")
print("对话 2:", response_2)
response_3 = conversation.predict(input="我的名字是?")
print("对话 3:", response_3) 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我叫李剑锋
AI:


C:\Users\76391\AppData\Local\Temp\ipykernel_2316\1408029818.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\76391\AppData\Local\Temp\ipykernel_2316\1408029818.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm,



> Finished chain.
对话 1: 你好，李剑锋！很高兴认识你。你的名字很有力量感呢，"剑锋"让人联想到锐利的剑刃，很有武侠风范！不知道你平时有什么兴趣爱好吗？比如喜欢看书、运动，或者对科技感兴趣？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我叫李剑锋
AI: 你好，李剑锋！很高兴认识你。你的名字很有力量感呢，"剑锋"让人联想到锐利的剑刃，很有武侠风范！不知道你平时有什么兴趣爱好吗？比如喜欢看书、运动，或者对科技感兴趣？
Human: 请问1+1等于多少?
AI:

> Finished chain.
对话 2: 1+1 是一个非常基础的数学问题呢！在常规的算术运算中，1+1 等于 2。不过，如果你是在脑筋急转弯或者某些特殊情境下提问，答案可能会不一样哦！但在这里，我就给你一个标准的数学答案吧：1+1=2！


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

### 2.2.2 RunnableWithMessageHistory（新版全量记忆）

#### 概念

RunnableWithMessageHistory 是 LangChain 新一代的对话历史管理工具。让模型在多轮对话中记住上下文，而不需要在每次调用时手动传入所有历史。

其对比 ConversationBufferMemory 的优势在于：
* 支持多会话管理：通过 session_id 管理不同用户或不同会话的历史，互不干扰。
* 解耦：历史记录和链逻辑分离，任何 Runnable 都能加记忆。
* 可扩展：可以替换存储层，比如存数据库、Redis，而不是内存变量。

#### 代码示例

首先我们像之前一样定义好提示词模版、模型以及将这两者组合在一起的 **LCEL** 链。但是这里和普通的提示词模版不一样的是，我们要在这个中间加上记忆过往记忆部分的内容，这里用的就是 `MessagesPlaceholder()` 方法，并且绑定好传入的名称。

然后呢我们还需要设置存储位置（`session_store`）及载入记忆的函数（`get_session_history`）。这个函数其实做的事情就是判定传入的 `session_id` 是否在存储的字典 `session_store` 是否存在（这个 `session_id` 是用于判定是否是同一段对话的关键内容），假如存在那就把记忆导出来传给提示词模版中的占位符 `MessagesPlaceholder()` ，假如不存在的话就在存储的字典里新建一条记录，也就是 `{"session_id" :ChatMessageHistory()}` 这样的格式。

再然后我们就需要通过 `RunnableWithMessageHistory` 将前面定义好的这些内容拼装起来，包括需要进行记忆的链（`runnable`）、传递记忆的方法（`get_session_histoy`）、输入信息的变量名称（`input_messages_key`）以及传入历史记录的变量名称（`history_messages_key`）。这样绑定过后，就能够实现记忆的自动存储及调用了。

最后我们还需要做的是设置一个单独的 `session_id` ，也就是这里的 "test_session1" 来作为这段对话独特的记忆标签。然后就可以正常的与大模型进行对话沟通了。但是这里需要注意的是，我们这里调用的就不是前面创建的 LCEL 链了，而是包裹了 LCEL 以及记忆的历史记录链 `RunnableWithMessageHistory` 。并且在调用的时候我们需要将 `session_id` 作为 config 传入。传入后每次当我们调用完获取回复，LangChain 就会根据我们 `session_id` 找到对应的历史记录，并把信息传入进去。后续我们可以通过打印的方式来对这部分历史的信息进行了解。

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os

llm = ChatOpenAI(
model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human", "{input}")])

chain = prompt | llm

session_store = {}

def get_session_history(session_id):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]


chain_with_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

session_id = "test_session1"

# 第一次调用
response1 = chain_with_history.invoke(
    {"input": "什么是人工智能, 50字介绍一下"},
    config={"configurable": {"session_id": session_id}}
)
print("第一次回答：", response1.content)

# 第二次调用，带历史
response2 = chain_with_history.invoke(
    {"input": "它有哪些应用, 50字介绍一下"},
    config={"configurable": {"session_id": session_id}}
)
print("第二次回答：", response2.content)


第一次回答： 人工智能是模拟人类智能的科学与技术，通过算法和模型让机器具备学习、推理、感知等能力，以解决复杂问题。
第二次回答： 人工智能应用广泛，涵盖智能语音助手、自动驾驶、医疗影像诊断、金融风控及个性化推荐等诸多领域。


## 2.3 记忆节省方法

### 2.3.1 使用原因

使用全量记忆虽然可以完整保留对话历史，但它也有一些显著的缺点或潜在问题，主要包括以下几点：

* 上下文限制：大模型有最大 token 限制（例如 8k、32k），超过就会报错。
* 使用成本增加：输入 token 越多，调用费用和响应时间越高。
* 大模型无法准确理解信息：冗长的上下文里可能掺杂大量无关信息，反而影响模型准确性。

因此，LangChain 提供了几种常见的“节省记忆”方法，比如：
* ConversationBufferWindowMemory（保存前 n 轮对话）
* ConversationTokenBufferMemory（保存 n 个 token 的对话内容）
* ConversationSummaryMemory（总结后保存）

但是由于这些方法都即将在 LangChain 的 1.0 版本时被丢弃，因此我们需要通过 RunnableWithMessageHistory + trim_messages （记忆剪辑） 的方法重新的实现。

### 2.3.2 trim_messages（记忆剪辑）

#### 概念

在对话记忆管理中，LangChain 提供了一个非常实用的工具函数 —— **`trim_messages`**。它的作用是对已有的对话历史进行“剪辑”，以便控制上下文长度，避免超过模型的最大 token 限制。并且，通过灵活的参数设置，我们可以：

* 控制上下文的最大长度
* 保证关键消息（如 system 指令）不丢失
* 根据需求选择合适的裁剪策略

它通常与 **RunnableWithMessageHistory** 结合使用，形成一个完整的对话记忆管理解决方案。

#### 基本用法

`trim_messages` 接收一组消息（Message 列表），以及一些裁剪参数，返回裁剪后的消息列表。

```python
from langchain_core.messages import trim_messages

trimmed = trim_messages(
    messages=history.messages,
    token_counter=lambda msgs: sum(len(str(m.content)) for m in msgs),
    max_tokens=100,
    start_on="human",   # 从 human 消息开始截断
    include_system=True, # 是否保留 system 提示
    strategy="last",    # 裁剪策略
    allow_partial=True   # 是否允许部分保留
)
```

#### 常见参数说明

* **messages**：需要裁剪的消息历史。
* **token_counter**：自定义 token 计数函数（例如按字符长度或真实 token 数）。
* **max_tokens**：上下文允许的最大 token 数。
* **start_on**：指定从哪种角色的消息开始裁剪（如 human / ai）。
* **include_system**：是否始终保留系统消息（system role）。
* **strategy**：裁剪策略，例如 `last` 表示保留最新消息。
* **allow_partial**：是否允许只保留消息的一部分。

#### 裁剪策略

1. **保留最新（last）**：只保留最近的若干消息，最常用。
2. **保留最早（first）**：保留开头的一些上下文（较少使用）。

### 2.3.3 ConversationBufferWindowMemory（保存前 n 轮对话）

#### 概念

ConversationBufferWindowMemory 是 LangChain 提供的一种“有限长度的对话记忆方式”，它不会保留完整对话历史，而是仅保留最近 N 轮消息（由用户设定）。

#### 工作机制

* 每轮对话（用户输入 + 模型输出）会被记录下来。
* 系统仅保留最近设定的 k 轮对话，其余部分会被“遗忘”。
* 每次调用模型时，仅发送这 N 轮内容作为 prompt 上下文。

#### 代码示例

那对于要保留前前 n 轮对话的记忆而言，我们记忆剪裁的方式就是先去根据 session_id 获取对应的完整记忆信息，然后从里面提取出对应的内容。那裁剪的话，本质上就是去看这里面有多少条信息（通过 `len()` 的方式去计算），然后只留下最近发生的 2 条聊天记录即可（一条 AIMessage 和 一条 HumanMessage）。那根据这个方法我们设计了 `trim_by_conversation_pairs()` 这个函数：

```python
def trim_by_conversation_pairs(history, max_pairs=1, include_system=True):
    """
    按对话段数裁剪历史，每段 = Human + AI 两条消息
    默认只保留最近 max_pairs 段
    """
    trimmed = trim_messages(
        history.messages, # 从保存的字典里获取信息列表
        token_counter=lambda msgs: len(msgs),  # 按消息条数统计（列表里有多少内容）
        max_tokens=max_pairs,  # 保留 max_pairs 段
        start_on="human",  # 确保每次传入模型的对话历史都从 HumanMessage 开始  
        include_system=include_system, # 保留系统提示词信息
        strategy="last",  # 保留最新的内容
    )
    history.messages[:] = trimmed  # 将剪裁好的信息更新
```

我们只需要传入对应的历史信息，然后就能够在传入到提示词模版中的记忆变量前，将记忆进行删减，从而节省其中所花费 token 的数量。最后我们设置一个新的 session_id ，并把记录打印出来。那这样就能看到只保留了最近的一条消息。

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages
import os

def trim_by_conversation_pairs(history, max_pairs=1, include_system=True):
    """
    按对话段数裁剪历史，每段 = Human + AI 两条消息
    默认只保留最近 max_pairs 段
    """
    trimmed = trim_messages(
        history.messages, # 从保存的字典里获取信息列表
        token_counter=lambda msgs: len(msgs),  # 按消息条数统计（列表里有多少内容）
        max_tokens=max_pairs,  # 保留 max_pairs 段
        start_on="human",  # 确保每次传入模型的对话历史都从 HumanMessage 开始  
        include_system=include_system, # 保留系统提示词信息
        strategy="last",  # 保留最新的内容
    )
    history.messages[:] = trimmed  # 将剪裁好的信息更新

session_store = {}
def get_session_history(session_id, max_pairs=1):
    """每次获取历史时自动裁剪，保证不会无限增长"""
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    history = session_store[session_id]
    trim_by_conversation_pairs(history, max_pairs=max_pairs)  # 按对话段数裁剪
    return history

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  MessagesPlaceholder(variable_name="chat_history"), 
  ("human", "{input}")])
chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
  runnable=chain,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history")

session_id = "test_session2"

# 第一次调用
response1 = chain_with_history.invoke(
  {"input": "什么是人工智能, 50字介绍一下"},
  config={"configurable": {"session_id": session_id}}
)
print("第一次回答：", response1.content)

# 第二次调用，带历史
response2 = chain_with_history.invoke(
  {"input": "它有哪些应用, 50字介绍一下"},
  config={"configurable": {"session_id": session_id}}
)
print("第二次回答：", response2.content)

# 打印裁剪后的历史
print("\n=== 裁剪后的会话历史（只保留最近一轮） ===")
for i, msg in enumerate(session_store[session_id].messages, 1):
    print(i, msg.type, msg.content)

第一次回答： 人工智能是模拟人类智能的计算机系统，能学习、推理、解决问题，在多领域展现强大能力并持续拓展应用边界。
第二次回答： 它（以AI为例）应用广泛，涵盖智能客服、医疗诊断、自动驾驶、教育辅导、金融风控等领域，提升效率与服务质量。

=== 裁剪后的会话历史（只保留最近一轮） ===
1 human 它有哪些应用, 50字介绍一下
2 ai 它（以AI为例）应用广泛，涵盖智能客服、医疗诊断、自动驾驶、教育辅导、金融风控等领域，提升效率与服务质量。


### 2.3.4 ConversationTokenBufferMemory（基于 Token 的记忆裁剪）

除了“保留前 n 轮对话”这种相对宽松的方式外，还有一种更严格的节省资源方法 —— **基于 Token 的记忆裁剪**。这种方法会限定对话历史最多只能占用固定数量的 **token**，一旦超过，就会自动丢弃最早的内容，从而保证上下文始终处于可控范围内。

这种机制过去在 LangChain 中对应的实现就是 **ConversationTokenBufferMemory**。它的特点是：

* 不关心“对话轮数”，只关心 token 总量。
* 当对话逐渐变长时，会优先保留最近的内容，把旧消息逐步丢弃。
* 更加适合对 **上下文长度** 和 **计算成本** 有严格限制的场景。

#### Token 的概念

要理解这种方式，我们需要先弄清楚 **token** 是什么。简单来说：

* **Token ≠ 字符**：在 NLP 模型里，一个 token 并不是单个汉字或字母，而是模型分词后的最小单位。
* **举例**：

  * 英文单词 *"apple"* 可能就是 1 个 token。
  * 中文“苹果”通常会被拆分为 1~2 个 token。
  * 较长的英文单词可能会被拆成多个 token。
* **意义**：token 是模型处理的最基本单元，模型的上下文限制、计算开销、调用费用都直接和 token 数量挂钩。

因此，ConversationTokenBufferMemory 的本质，就是通过严格限制 **token 数量**，来控制上下文大小，避免超出模型的处理上限。


#### 结合 RunnableMemory 与 trim_messages 的实现

在新版 LCEL 框架下，我们可以通过 **RunnableMemory** 搭配 `trim_messages` 来实现类似的功能。


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages
import os

# 定义 token 计算方法（简单认定一个字符为1个 token）
def count_tokens_by_len(msgs):
    return sum(len(str(m.content)) for m in msgs)

def trim_history_inplace(history, max_tokens=60):
    trimmed = trim_messages(
        history.messages, # 传入历史信息
        token_counter=count_tokens_by_len, # 按一个字符的方式计算每一段内容的 token
        max_tokens=max_tokens, # 传入的最大 token 数量
        start_on="human", # 确保每次传入模型的对话历史都从 HumanMessage 开始
        include_system=True, # 包含系统提示词
        strategy="last",) # 保留最新的内容
    history.messages[:] = trimmed  # 原地更新

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  MessagesPlaceholder(variable_name="chat_history"), 
  ("human", "{input}")])
chain = prompt | llm

session_store = {}

def get_session_history(session_id, max_tokens=60):
  if session_id not in session_store:
    session_store[session_id] = ChatMessageHistory()
  history = session_store[session_id]
  trim_history_inplace(history, max_tokens=max_tokens) # 每次取之前自动裁剪
  return history

chain_with_history = RunnableWithMessageHistory(
  runnable=chain,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history")

session_id = "test_session3"

response1 = chain_with_history.invoke(
  {"input": "什么是人工智能, 50字介绍一下"},
  config={"configurable": {"session_id": session_id}})
print("第一次回答：", response1.content)

response2 = chain_with_history.invoke(
  {"input": "它有哪些应用, 100字介绍一下"},
  config={"configurable": {"session_id": session_id}})
print("第二次回答：", response2.content)

# 打印裁剪后的历史
print("\n=== 裁剪后的会话历史（只保留最新的 n 个 token） ===")
for i, msg in enumerate(session_store[session_id].messages, 1):
  print(i, msg.type, msg.content)

第一次回答： 人工智能是模拟人类智能的科技，能通过学习、推理等自主处理信息，在多领域实现智能决策与任务执行。
第二次回答： 它若指人工智能，应用广泛。在医疗领域辅助诊断疾病、制定治疗方案；教育上实现个性化学习规划；交通中助力自动驾驶；金融里进行风险评估与智能投顾；工业里优化生产流程、质量检测；还能用于智能客服、智能家居等，极大改变生活与工作模式。

=== 裁剪后的会话历史（只保留最新的 n 个 token） ===
1 human 它有哪些应用, 100字介绍一下
2 ai 它若指人工智能，应用广泛。在医疗领域辅助诊断疾病、制定治疗方案；教育上实现个性化学习规划；交通中助力自动驾驶；金融里进行风险评估与智能投顾；工业里优化生产流程、质量检测；还能用于智能客服、智能家居等，极大改变生活与工作模式。


#### 思路解析

* 这里我们自定义了 `count_tokens_by_len` 来估算 token 数量（虽然和真实分词略有差距，但是可以非常直观的进行展示）。
* `trim_history_inplace` 每次都会检查当前的对话历史，并在必要时执行“原地裁剪”。
* 这种实现方式能保证 **上下文不会无限膨胀**，而且可以灵活调整 `max_tokens` 的阈值。

结合 **RunnableWithMessageHistory** 使用时，这样的剪裁逻辑就能在每次调用前自动生效，从而确保上下文始终保持在安全范围之内。

### 2.3.5 ConversationSummaryMemory（总结后保存）

除了“保留最近几轮”或“按 token 数量裁剪”以外，还有一种更智能的记忆方式 —— **ConversationSummaryMemory**。它的核心思路是：

* 当对话逐渐变长时，不是简单地丢弃旧内容；
* 而是通过 **大模型生成摘要** 的方式，把历史信息压缩成简短概述，然后再保存下来。

这种方式的优势在于：

* **信息保留更完整**：即使丢弃了原始消息，重要的事实与上下文依然能通过摘要继续存在。
* **上下文更紧凑**：摘要比原文短得多，大幅减少 token 占用。
* **逻辑更清晰**：通过模型生成的总结，能去除冗余，保留核心脉络。


#### 工作机制

1. 用户与模型的对话不断积累。
2. 当历史过长时，触发总结逻辑。
3. 模型会把早期的对话“压缩”为一段摘要（例如“用户主要在咨询旅游路线和机票信息”）。
4. 旧的详细消息被丢弃，摘要替代它进入记忆中。
5. 后续的调用会在上下文中继续携带这个摘要，从而保持对历史的理解。

#### RunnableWithMessageHistory + 自定义剪裁

因为 trim_messages 方法无法完成先总结后保存的任务，因此在新版 LCEL 的语境下，我们通过 **RunnableWithMessageHistory** 搭配自定义逻辑来实现“总结后保存”的功能。下面给出一个完整的示例代码，演示如何在每次对话后，对最后一条 AI 回复进行摘要，从而节省上下文。




In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage
import os

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  MessagesPlaceholder(variable_name="chat_history"), 
  ("human", "{input}")])
chain = prompt | llm

def summarize_text_with_llm(text: str, max_len: int = 50) -> str:
    """用大模型总结文本，控制在 max_len 字符内"""
    
    prompt = ChatPromptTemplate.from_template(
        "请在不超过{max_len}个字内，简要总结以下内容：\n\n{text}"
    )

    # 调用大模型，传入替换好参数的提示词，返回一个消息对象或字符串
    summary = llm.invoke(prompt.format(text=text, max_len=max_len))

    return summary.content # 返回文本内容

session_store = {}
def get_session_history(session_id: str, summary_max_len: int = 50):
    """获取会话历史，并将最后一条AIMessage总结后替换"""
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    history = session_store[session_id]
    # 如果会话里有消息，且最后一条消息是 AI 回复，就做总结
    if history.messages and isinstance(history.messages[-1], AIMessage):
        last_ai_msg = history.messages[-1] # 取出最后一条 AI 回复
        summarized = summarize_text_with_llm(last_ai_msg.content, max_len=summary_max_len) # 调用大模型进行总结，控制在 summary_max_len 字符内
        history.messages[-1] = AIMessage(content=summarized)  # 替换为总结内容
    return history # 返回更新后的历史记录

chain_with_history = RunnableWithMessageHistory(
  runnable=chain,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history")

session_id = "test_session4"

response1 = chain_with_history.invoke(
  {"input": "什么是人工智能, 50字介绍一下"},
  config={"configurable": {"session_id": session_id}})
print("第一次回答：", response1.content)

response2 = chain_with_history.invoke(
  {"input": "它有哪些应用, 50字介绍一下"},
  config={"configurable": {"session_id": session_id}})
print("第二次回答：", response2.content)

# 打印裁剪后的历史
print("\n=== 总结后的会话历史 ===")
for i, msg in enumerate(session_store[session_id].messages, 1):
  print(i, msg.type, msg.content)

第一次回答： 人工智能是模拟人类智能的计算机系统，能学习、推理、解决问题，涵盖机器学习、自然语言处理等技术，改变生活工作方式。
第二次回答： 人工智能应用广泛，如智能语音助手、自动驾驶、医疗影像诊断，提升效率，革新多行业服务模式。

=== 总结后的会话历史 ===
1 human 什么是人工智能, 50字介绍一下
2 ai 人工智能模拟人类智能，涵盖机器学习等技术，能学习推理，改变生活工作方式。
3 human 它有哪些应用, 50字介绍一下
4 ai 人工智能应用广泛，如智能语音助手、自动驾驶、医疗影像诊断，提升效率，革新多行业服务模式。


#### 思路解析

1. **链的构建**：用 `ChatPromptTemplate` + `ChatOpenAI` 构建基本对话链。
2. **摘要逻辑**：自定义 `summarize_text_with_llm`，调用 LLM 对 AI 的回复做简短总结。
3. **历史管理**：在 `get_session_history` 中，每次更新历史时，用摘要替换最后一条 AI 回复。
4. **RunnableWithMessageHistory**：把链与记忆机制结合，让它在多轮对话中自动调用摘要逻辑。

通过这种方式，我们实现了类似 **ConversationSummaryMemory** 的效果：在保持上下文连贯的同时，大幅度减少了历史消息的冗余。

# 3. Gradio 前端记忆

假如我们希望能够把聊天记录显示在gradio上，我们就需要Gradio.Chatbot这个组件！

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import gradio as gr
import os

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human", "{input}")])

chain = prompt | llm

session_store = {}

def get_session_history(session_id):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]


chain_with_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

session_id = "session_test5"
def chat(message, history):
    history = history or []
    resp = chain_with_history.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}},
    )
    reply = resp.content if hasattr(resp, "content") else str(resp)
    history = history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": reply},
    ]
    return "", history

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 聊天机器人（内置记忆）")
    chatbot = gr.Chatbot(label="LangChain ChatBot")
    msg = gr.Textbox(placeholder="和我聊聊吧...", lines=1)
    msg.submit(chat, [msg, chatbot], [msg, chatbot])
    gr.ClearButton([msg, chatbot])

demo.launch()

c:\Users\76391\.conda\envs\chapter_3_2025_12_15\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


或者使用前面说的 ChatInterface 实现，此时会更加简化一些。

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os
import gradio as gr

def get_session_history(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

session_store = {}

def llm_response(content, history):
    llm = ChatOpenAI(
        model="ernie-3.5-8k",
        openai_api_key=os.environ.get("OPENAI_API_KEY"),
        base_url="https://aistudio.baidu.com/llm/lmapi/v3",
    )
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ])
    
    chain = prompt | llm
    
    session_id = "user_1"
    
    chain_with_history = RunnableWithMessageHistory(
        runnable=chain,
        get_session_history=get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )
    resp = chain_with_history.invoke(
            {"input": content},
            config={"configurable": {"session_id": session_id}},
        )
    reply = resp.content if hasattr(resp, "content") else str(resp)
    return reply

demo = gr.ChatInterface(fn=llm_response)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
